# Taller Despliegue Flask API, PARTE I: Local en CSV

Para esta primera parte vamos a desplegar un modelo de machine learning en una API **para su consumo en local** (como hemos visto en el taller anterior para la base de datos de novelas de ciencia ficción). 

Entrenaremos un modelo, lo guardaremos entrenado, y desarrollaremos una API que permita consumir dicho modelo desde cualquier otra tecnología, pero primero en un servico local.

Para que tengas más contexto... **Se presenta el siguiente caso de uso**

Una empresa distribuidora de muebles del ámbito nacional pretende utilizar un modelo desarrollado por el departamento de analítica, con el que consiguen una predicción de las ventas a partir de los gastos en marketing de anuncios en televisión, radio y periódicos. Quieren incorporar estos datos dentro de su página web interna, donde comparten todo tipo de información relativa a resultados de la empresa, ventas, adquisiciones, etc... La web está desarrollada en AngularJS, mientras que el modelo se desarrolló en Python, por lo que precisamos de una interfaz de comunicación entre ambos sistemas.

El equipo de desarrollo necesita que implementes un microservicio para que ellos puedan consumir el modelo desde la propia web. El microservicio tiene que cumplir las siguientes características:
1. Ofrezca la predicción de ventas a partir de todos los valores de gastos en publicidad.
2. Podamos actualizar la base de datos con nuevos registros, una vez conozcamos los valores de venta reales.
3. Posibilidad de reentrenar el modelo con los nuevos registros.

**¿Qué es necesario implementar, y se deja como ejercicio opcional?**  

1. Por simplicidad del ejercicio, la base de datos con la que se trabaja es un CSV ("Advertising.csv")

2. El modelo. En el momento de reentrenamiento, simplemente se entrena la regresión lineal con los últimos datos. No es necesario comprobar si los resultados son mejores o peores, tenemos missings u outliers...

3. Queremos implementar 3 endpoints: El del mensaje local para el acceso al /, un endpoint que dados los valores de inversión en TV, Radio y Periódicos nos diga las ventas esperadas y otro endpoint en el que el servidor de la API compruebe si tiene datos nuevos y reentrene el modelo y vuelva a calcular sus métricas. (Suponemos que el fichero)

**NOTA**: Cuentas con un script de Python (*model.py*) con el código de entrenamiento del modelo ya hecho, puesto que el desarrollo de un modelo de machine learning no es el objetivo del ejercicio, sino el diseño de una API con Flask

### Procedimiento

**#1. Creación del modelo**

Carga el script `model.py`, revisalo y ejecútalo para crear nuestro modelo.

**#2. Creación del script para dar servicio a la API (modo local)**

A partir de los scripts del taller anterior con Flask crea un nuevo script en el que añadas el routing a las siguientes funciones (revísalas antes) (en los comentarios de cada función se describe el endpoint)

```python
def hello(): # Ligado al endopoint "/" o sea el home, con el método GET
    return "Bienvenido a mi API del modelo advertising"
```

```python
def predict(): # Ligado al endpoint '/api/v1/predict', con el método GET

    model = pickle.load(open('ad_model.pkl','rb'))
    tv = request.args.get('tv', None)
    radio = request.args.get('radio', None)
    newspaper = request.args.get('newspaper', None)

    print(tv,radio,newspaper)
    print(type(tv))

    if tv is None or radio is None or newspaper is None:
        return "Args empty, the data are not enough to predict"
    else:
        prediction = model.predict([[float(tv),float(radio),float(newspaper)]])
    
    return jsonify({'predictions': prediction[0]})
```

Para la función anterior, ¿Cómo crees que espera los argumentos la API? (¿como parámetros en el cuerpo de la petición, como querystring,...?)

```python
def retrain(): # Rutarlo al endpoint '/api/v1/retrain/', metodo GET
    if os.path.exists("data/Advertising_new.csv"):
        data = pd.read_csv('data/Advertising_new.csv')

        X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['sales']),
                                                        data['sales'],
                                                        test_size = 0.20,
                                                        random_state=42)

        model = Lasso(alpha=6000)
        model.fit(X_train, y_train)
        rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
        mape = mean_absolute_percentage_error(y_test, model.predict(X_test))
        model.fit(data.drop(columns=['sales']), data['sales'])
        pickle.dump(model, open('ad_model.pkl', 'wb'))

        return f"Model retrained. New evaluation metric RMSE: {str(rmse)}, MAPE: {str(mape)}"
    else:
        return f"<h2>New data for retrain NOT FOUND. Nothing done!</h2>"

```

**#3. Hora de ejecutar y probar varias predicciones y el endpoint de retrain**